In [1]:
root_folder = "../results"

In [2]:
metrics2paper = {
    "R2": "R2",
    "RMSE": "RMSE",
    "NRMSE": "%RMSE",
}

In [3]:
import pandas as pd
from glob import glob
from experiments_to_run import *

paths = glob(f'{root_folder}/metrics*')

df = []
for path in paths:
    tmp_df = pd.read_pickle(path)
    df.append(tmp_df)
df = pd.concat(df)

metrics = ["R2", "RMSE", "NRMSE"]
mean_df = df.groupby(["target", "config", "model"]).agg("mean")[metrics]#.reset_index()
std_df = df.groupby(["target", "config", "model"]).agg("std")[metrics]#.reset_index()

std_df.columns = ["s_"+x for x in std_df.columns]
overall_df = pd.concat((mean_df, std_df), axis=1)

for metric in metrics:
    overall_df[f"str_{metric}"] = overall_df.apply(lambda row: f"{row[metric]:.2f} ± {row['s_'+metric]:.2f}", axis=1)
    
overall_df = overall_df.drop(columns = [x for x in overall_df.columns if "str" not in x])
overall_df.columns = metrics
overall_df = overall_df.reset_index()

overall_df.sort_values(["target","config", "model"], inplace=True)
overall_df.rename(columns=metrics2paper, inplace=True)
overall_df

,target,config,model,R2,RMSE,%RMSE
0,CS,Conf1,CatBoost,0.28 ± 0.06,51.15 ± 3.64,66.13 ± 6.34
1,CS,Conf1,GBDT,0.25 ± 0.04,52.50 ± 4.04,67.90 ± 7.03
2,CS,Conf1,KNN,0.23 ± 0.05,52.96 ± 3.13,68.45 ± 5.74
3,CS,Conf1,MLP,0.12 ± 0.10,56.86 ± 5.58,73.52 ± 8.78
4,CS,Conf1,RF,0.30 ± 0.07,50.53 ± 3.83,65.32 ± 6.54
5,CS,Conf1,SVR,0.20 ± 0.05,54.28 ± 4.82,70.23 ± 8.28
6,CS,Conf1,XGBoost,0.21 ± 0.12,53.49 ± 3.43,69.09 ± 5.55
7,CS,Conf2,CatBoost,0.29 ± 0.07,50.71 ± 3.29,65.55 ± 5.87
8,CS,Conf2,GBDT,0.25 ± 0.05,52.27 ± 4.00,67.60 ± 6.95
9,CS,Conf2,KNN,0.26 ± 0.05,52.06 ± 3.46,67.30 ± 6.21


In [4]:
for target in TARGETS:
    tmp = overall_df[(overall_df.target==target)].drop(columns=["target"])
    print("results exported to", f"{root_folder}/table_{target}.[txt/xlsx]")
    tmp.to_markdown(f"{root_folder}/table_{target}.txt", index=False, tablefmt="fancy_grid")
    tmp.to_excel(f"{root_folder}/table_{target}.xlsx", index=False)
    
    print("results exported to", f"../figures_and_tables/table_performance_{target}.[txt/xlsx/csv]")
    tmp.to_markdown(f"../figures_and_tables/table_performance_{target}.txt", index=False, tablefmt="fancy_grid")
    tmp.to_excel(f"../figures_and_tables/table_performance_{target}.xlsx", index=False)
    tmp.to_csv(f"../figures_and_tables/table_performance_{target}.csv", index=False)

results exported to ../results/table_CS.[txt/xlsx]
results exported to ../figures_and_tables/table_performance_CS.[txt/xlsx/csv]
results exported to ../results/table_CSE.[txt/xlsx]
results exported to ../figures_and_tables/table_performance_CSE.[txt/xlsx/csv]
